In [10]:
from pydantic import BaseModel,Field
from langchain_google_genai import ChatGoogleGenerativeAI

class Country(BaseModel):
    """Information about a country. """
    name: str = Field(description = "name of the country")
    language: str = Field(description = "language of the country")
    capital: str = Field(description = "Capital of the country")

from dotenv import load_dotenv
load_dotenv()

llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash")
structured_llm = llm.with_structured_output(Country)
structured_llm

RunnableBinding(bound=ChatGoogleGenerativeAI(model='models/gemini-2.5-flash', google_api_key=SecretStr('**********'), client=<google.ai.generativelanguage_v1beta.services.generative_service.client.GenerativeServiceClient object at 0x7c938c0e7e90>, default_metadata=()), kwargs={'tools': [{'type': 'function', 'function': {'name': 'Country', 'description': 'Information about a country. ', 'parameters': {'properties': {'name': {'description': 'name of the country', 'type': 'string'}, 'language': {'description': 'language of the country', 'type': 'string'}, 'capital': {'description': 'Capital of the country', 'type': 'string'}}, 'required': ['name', 'language', 'capital'], 'type': 'object'}}}], 'tool_choice': 'Country'}, config={}, config_factories=[])
| PydanticToolsParser(first_tool_only=True, tools=[<class '__main__.Country'>])

In [11]:
structured_llm.invoke("Sri Lanlkla")

Country(name='Sri Lanka', language='Sinhala', capital='Sri Jayawardenepura Kotte')

In [ ]:
from typing_extensions import Annotated, TypedDict
from typing import Optional


# TypedDict
class Joke(TypedDict):
    """Joke to tell user."""

    setup: Annotated[str, ..., "The setup of the joke"]

    # Alternatively, we could have specified setup as:

    # setup: str                    # no default, no description
    # setup: Annotated[str, ...]    # no default, no description
    # setup: Annotated[str, "foo"]  # default, no description

    punchline: Annotated[str, ..., "The punchline of the joke"]
    rating: Annotated[Optional[int], None, "How funny the joke is, from 1 to 10"]

llm = ChatGoogleGenerativeAI(model="openai")

structured_llm = llm.with_structured_output(Joke)

structured_llm.invoke("Tell me a joke about cats")

ValueError: no signature found for builtin type <class 'dict'>

In [15]:
from typing import Optional
from pydantic import BaseModel, Field
from langchain_google_genai import ChatGoogleGenerativeAI

class Joke(BaseModel):
    setup: str = Field(..., description="The setup of the joke")
    punchline: str = Field(..., description="The punchline of the joke")
    rating: Optional[int] = Field(
        default=None,
        description="How funny the joke is, from 1 to 10",
    )

llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash")

structured_llm = llm.with_structured_output(
    schema=Joke.model_json_schema(),  # <-- JSON schema, not the class itself
    method="json_schema",             # <-- avoids tool/function path
)

resp = structured_llm.invoke("Tell me a joke about cats")
print(resp)  # dict with keys: setup, punchline, rating


ValueError: Received unsupported arguments {'method': 'json_schema'}

In [13]:

json_schema = {
    "title": "joke",
    "description": "Joke to tell user.",
    "type": "object",
    "properties": {
        "setup": {
            "type": "string",
            "description": "The setup of the joke",
        },
        "punchline": {
            "type": "string",
            "description": "The punchline to the joke",
        },
        "rating": {
            "type": "integer",
            "description": "How funny the joke is, from 1 to 10",
            "default": None,
        },
    },
    "required": ["setup", "punchline"],
}
structured_llm = llm.with_structured_output(json_schema)

structured_llm.invoke("Tell me a joke about cats")

Key 'parameters' is not supported in schema, ignoring


[{'args': {'punchline': 'Because they make up everything!',
   'setup': "Why don't scientists trust atoms?"},
  'type': 'joke'}]